In [2]:
import tensorflow as tf

model = tf.keras.models.load_model('model/checkpoint.h5')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization_8 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
activation_8 (Activation)    (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
batch_normalization_9 (Batch (None, 24, 24, 128)      

In [3]:
import cv2
import numpy as np

video = cv2.VideoCapture(0)
frames = 0

face_cascade = cv2.CascadeClassifier()
face_cascade.load('haarcascade_frontalface_default.xml')
label_map = ['Anger', 'Neutral', 'Fear', 'Happy', 'Sad', 'Surprise']

while True:
    check, frame = video.read()
    
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.05, minNeighbors=5)
    for x, y, w, h in faces:
        if frames % 5 == 0:
            face = frame[y:y+h, x:x+w,]
            face_g = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            face_g_r = cv2.resize(face_g, (48, 48))
            labels = model.predict(face_g_r.reshape(1, 48, 48, 1)/255.0)
            confidence = int(np.max(labels)*100)
            label = np.argmax(labels)
        frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cv2.putText(frame, str(confidence) + "% " + label_map[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    
    cv2.imshow("video", frame)
    frames += 1
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
        
video.release()
cv2.destroyAllWindows()